01 Working File

In [1]:
import numpy as np
import plotly.graph_objects as go

from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import sma_tv, bb_tv

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=3000,
)
timestamps = candles[:, CandleBodyType.Timestamp]
datetimes = timestamps.astype("datetime64[ms]")
open_prices = candles[:, CandleBodyType.Open]
high_prices = candles[:, CandleBodyType.High]
low_prices = candles[:, CandleBodyType.Low]
close_prices = candles[:, CandleBodyType.Close]
volume = candles[:, CandleBodyType.Volume]

In [32]:
source_sma = close_prices
length_sma = 20
sma_lookback = 5
sma_lookback -= 1

sma = sma_tv(source=source_sma, length=length_sma)

In [33]:
source_bb = close_prices
multi_bb = 2.5
length_bb = 20

basic_bb, upper_bb, lower_bb = bb_tv(length=length_bb, multi=multi_bb, source=source_bb)

In [4]:
down_trend = np.full_like(close_prices, False, dtype=np.bool_)
for i in range(sma_lookback, sma.size):
    down_trend[i] = (sma[i - sma_lookback : i + 1] < source_sma[i - sma_lookback : i + 1]).all()
bb_short_signals = close_prices > upper_bb
entries = (down_trend == True) & (bb_short_signals == True)
entry_signals = np.where(entries, close_prices, np.nan)

In [6]:
for i in range(sma_lookback, sma.size):
    down_trend[i] = (sma[i - sma_lookback : i + 1] < source_sma[i - sma_lookback : i + 1]).all()

In [10]:
bb_short_signals = close_prices > upper_bb

In [26]:
entries = (down_trend == True) & (bb_short_signals == True)
entry_signals = np.where(entries, close_prices, np.nan)

In [31]:
ul_rgb: str = "48, 123, 255"
basis_color_rgb: str = "255, 176, 0"
fig = go.Figure()
fig.add_candlestick(
    x=datetimes,
    open=open_prices,
    high=high_prices,
    low=low_prices,
    close=close_prices,
    name="Candles",
)
fig.add_scatter(
    x=datetimes,
    y=sma,
    line_color="aqua",
)
fig.add_scatter(
    x=datetimes,
    y=lower_bb,
    name="lower",
    line_color=f"rgb({ul_rgb})",
),
fig.add_scatter(
    x=datetimes,
    y=upper_bb,
    name="upper",
    line_color=f"rgb({ul_rgb})",
    fillcolor=f"rgba({ul_rgb}, 0.07)",
    fill="tonexty",
),
fig.add_scatter(
    x=datetimes,
    y=basic_bb,
    name="basis",
    line_color=f"rgb({basis_color_rgb})",
),
fig.add_scatter(
    x=datetimes,
    y=entry_signals,
    name="Signals",
    mode="markers",
    marker=dict(
        size=12,
        symbol="hexagram",
        color="yellow",
        line=dict(width=1, color="DarkSlateGrey"),
    ),
)
fig.update_layout(height=800, xaxis_rangeslider_visible=False)
fig.show()